# 正規化層

*Normalization Layers*

入力されたデータを正規化して返す層。

正規化する軸方向に依って色々な種類がある。

- バッチ正規化
- 層正規化
- インスタンス正規化
- グループ正規化

pytorchで挙動を見ながら理解していこう。  
- https://pytorch.org/docs/stable/nn.html#normalization-layers

### 用語

- サンプル: バッチ内の各データ
- 特徴量: サンプルを構成する各スカラー
- チャンネル: サンプルを構成するテンソルの1番上の次元。サンプル内の各データ。
- インスタンス: チャンネル内の各データ。

In [1]:
import torch
import torch.nn as nn


---

## バッチ正規化

*Batch Normalization*

[1] Ioffe, Sergey, and Christian Szegedy. "Batch normalization: Accelerating deep network training by reducing internal covariate shift." International conference on machine learning. pmlr, 2015.

バッチ内を正規化し、学習したパラメータに従ってスケーリング・シフトして出力する層。バッチ内は特徴量ごとに正規化する。

バッチサイズ$m$のミニバッチ$\mathcal B = \{\boldsymbol x_1,\boldsymbol x_2,\cdots,\boldsymbol x_m \}$が得られた時、以下の演算で出力値$\boldsymbol y_i$を決定する。

$$
\begin{align}

\boldsymbol\mu_{\mathcal B} &= \frac{1}{m}\sum_{i=1}^m \boldsymbol x_i \\
\boldsymbol\sigma^2_{\mathcal B} &= \frac{1}{m}\sum_{i=1}^m (\boldsymbol x_i - \boldsymbol\mu_{\mathcal B})^2 \\

\hat{\boldsymbol x}_i &= \frac{\boldsymbol x_i - \boldsymbol\mu_{\mathcal B}}{\sqrt{\boldsymbol\sigma^2_{\mathcal B} + \epsilon}} \\

\boldsymbol y_i &= \boldsymbol\gamma\hat{\boldsymbol x}_i + \boldsymbol\beta

\end{align}
$$
- $\boldsymbol x_i, \, \hat{\boldsymbol x}_i, \, \boldsymbol y_i, \, \boldsymbol\mu_{\mathcal B}, \, \boldsymbol\sigma^2_{\mathcal B}, \, \boldsymbol\gamma, \, \boldsymbol\beta \in \R^d$
- $d$: 特徴量の数
- $\epsilon$: 微小値（0除算回避用）

まずミニバッチ$\mathcal B$の平均$\boldsymbol\mu_{\mathcal B}$と分散$\boldsymbol\sigma^2_{\mathcal B}$を求める。次にそれらを用いて$\boldsymbol x_i$を正規化する。最後に$\boldsymbol\gamma,\boldsymbol\beta$を用いてスケーリングとシフトを行う。$\boldsymbol\gamma,\boldsymbol\beta$は学習可能なパラメータで、出力データ$\boldsymbol y_i$の分散と平均を意味する。まとめると、この層は、分布（分散$\boldsymbol\gamma$、平均$\boldsymbol\beta$）を学習し、その分布に従うように入力データを変換する層ということ。

さて、上記の演算は学習時に行うもので、推論時には使えない。ミニバッチ内の他のデータに依って出力が変わってしまうため。推論時は、$\boldsymbol y_i$は$\boldsymbol x_i$のみに依存している必要がある。また推論時はバッチサイズが1であることも多く、その場合$\hat{\boldsymbol x}_i$が$\boldsymbol 0$になるため$\boldsymbol y_i$が$\boldsymbol\beta$に固定されてしまう。

推論時は以下のような演算を行う。

$$
\hat{\boldsymbol x} = \frac{\boldsymbol x - \mathbb E[\boldsymbol x]}{\sqrt{\text{Var}[\boldsymbol x] + \epsilon}} \\
$$

入力データ$\boldsymbol x$の平均$\mathbb E[\boldsymbol x]$と分散$\text{Var}[\boldsymbol x]$を用いる。これらは学習時に観測したデータから求めることになるため、実質的に学習データ全体の平均と分散となる。

### CNNでのバッチ正規化

画像や畳み込み層からの出力は3次元のデータで表される。当然これらも同じように正規化することが可能である。サンプルの形状が`(c, h, w)`の場合、$c\times h\times w$個ずつパラメータ（平均、分散）を用意し、特徴量ごとに正規化（&スケーリング・シフト）するということ。

ただこの方法は基本的に使わず、実際はチャンネルごとに正規化する。サンプルを跨いだ同じ特徴マップの値を全て同じ種類の特徴量とみなし、それらを同時に正規化する。パラメータは平均と分散がチャンネルの数だけ必要になる。

確かに、各チャンネル各ピクセルの値を獨立した別々の特徴量として扱うのは違和感があるので、チャンネルごとに正規化するのは自然に感じる。論文[1]には

> For convolutional layers, we additionally want the normalization to obey the convolutional property (畳み込みレイヤーの場合、さらに正規化は畳み込みの性質に従うようにしたい。(Deepl訳))

と書いてあった。

### 実装

PyTorchで実装して挙動を確認してみよう。

まず学習時の挙動を確認する。適当なミニバッチを用意する。

In [2]:
num_features = 4
batch_size = 3
x = torch.arange(0, num_features * batch_size, dtype=torch.float32)
x = x.reshape(batch_size, num_features)
x

tensor([[ 0.,  1.,  2.,  3.],
        [ 4.,  5.,  6.,  7.],
        [ 8.,  9., 10., 11.]])

パラメータも初期化しておく。

In [3]:
gamma = torch.ones(num_features)
beta = torch.zeros(num_features)
eps = 1e-5

平均は全て0、分散は全て1で初期化した。

では正規化層の演算を実装する。まずミニバッチの統計量を求める。

In [4]:
mean = x.mean(dim=0)
var = x.var(dim=0)
print("平均:", mean)
print("分散:", var)

平均: tensor([4., 5., 6., 7.])
分散: tensor([16., 16., 16., 16.])


`dim=0`でバッチ軸を指定し、特徴量ごとの平均と分散を求めた。なお、本当は`x.var(dim=0, unbiased=False)`として標本分散を求めないといけない。ただ不偏分散の方が分かりやすいため、ここではそうしちゃう。

次は正規化。

In [5]:
x_hat = (x - mean) / torch.sqrt(var + eps)
x_hat

tensor([[-1.0000, -1.0000, -1.0000, -1.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000],
        [ 1.0000,  1.0000,  1.0000,  1.0000]])

平均0、分散1になった。

最後にパラメータでスケーリングとシフトを行う。

In [6]:
y = gamma * x_hat + beta
y

tensor([[-1.0000, -1.0000, -1.0000, -1.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000],
        [ 1.0000,  1.0000,  1.0000,  1.0000]])

今はパラメータが平均0、分散1なので変化なし。

以上がバッチ正規化の演算である。これを`nn.Module`として実装してみよう。

In [7]:
class BatchNormalization(nn.Module):
    def __init__(self, num_features, eps=1e-5):
        super().__init__()
        self.eps = eps
        self.gamma = nn.Parameter(torch.ones(num_features))
        self.beta = nn.Parameter(torch.zeros(num_features))

    def forward(self, x):
        mean = x.mean(dim=0)
        var = x.var(dim=0)
        x_hat = (x - mean) / (torch.sqrt(var) + self.eps)
        y = x_hat * self.gamma + self.beta
        return y

このように使う。

In [8]:
bn = BatchNormalization(num_features)
y = bn(x)
y

tensor([[-1.0000, -1.0000, -1.0000, -1.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000],
        [ 1.0000,  1.0000,  1.0000,  1.0000]], grad_fn=<AddBackward0>)

これで、簡易バッチ正規化層の完成。

さて、ちゃんとしたバッチ正規化層も作ってみよう。推論時の挙動を追加する。

In [9]:
class BatchNormalization(nn.Module):
    def __init__(self, num_features, eps=1e-5, momentum=0.1):
        super().__init__()
        self.eps = eps
        self.momentum = momentum
        self.gamma = nn.Parameter(torch.ones(num_features))
        self.beta = nn.Parameter(torch.zeros(num_features))
        self.register_buffer("running_mean", torch.zeros(num_features))
        self.register_buffer("running_var", torch.ones(num_features))

    def forward(self, x):
        if self.training: # 学習
            mean = x.mean(dim=0)
            var = x.var(dim=0, unbiased=False)
            var_unbiased = x.var(dim=0, unbiased=True)
            self.running_mean = (1 - self.momentum) * self.running_mean + self.momentum * mean
            self.running_var = (1 - self.momentum) * self.running_var + self.momentum * var_unbiased
        else: # 推論
            mean = self.running_mean
            var = self.running_var
        x_hat = (x - mean) / (torch.sqrt(var) + self.eps)
        y = x_hat * self.gamma + self.beta
        return y

$\mathbb E[\boldsymbol x]$、$\text{Var}[\boldsymbol x]$を`running_mean`、`running_var`として保持し、推論時に使う。そしてそれらは学習時に都度更新する。移動平均によって動的に求めている。$\text{Var}[\boldsymbol x]$は不偏分散なので`unbiased=True`にする。

学習モードで適当にデータを見せると`running_mean`、`running_var`が更新される。

In [10]:
# 初期値
bn = BatchNormalization(num_features)
bn.state_dict()

OrderedDict([('gamma', tensor([1., 1., 1., 1.])),
             ('beta', tensor([0., 0., 0., 0.])),
             ('running_mean', tensor([0., 0., 0., 0.])),
             ('running_var', tensor([1., 1., 1., 1.]))])

In [11]:
bn.train()
torch.manual_seed(0)
for _ in range(100):
    x = torch.randn(batch_size, num_features)
    bn(x)
bn.state_dict()

OrderedDict([('gamma', tensor([1., 1., 1., 1.])),
             ('beta', tensor([0., 0., 0., 0.])),
             ('running_mean', tensor([ 0.0527, -0.0417, -0.0876, -0.0537])),
             ('running_var', tensor([0.9879, 0.4089, 1.3890, 0.7146]))])

推論モードにすると`running_mean`、`running_var`が使われる。

In [12]:
bn.eval()
x = torch.randn(1, num_features)
y = bn(x)
y

tensor([[ 0.8423, -1.5081,  1.0178,  2.3462]], grad_fn=<AddBackward0>)

こういうこと。

In [13]:
gamma, beta, running_mean, running_var = bn.state_dict().values()
eps = bn.eps
y = (x - running_mean) / torch.sqrt(running_var + eps) * gamma + beta
y

tensor([[ 0.8423, -1.5081,  1.0178,  2.3462]])

### PyTorchに実装されている層

先ほど実装した`BatchNormalization`は`nn.BatchNorm1d`としてそのままPyTorchに実装されている。
- https://pytorch.org/docs/stable/generated/torch.nn.BatchNorm1d.html

挙動もほぼ同じ。

In [14]:
bn = nn.BatchNorm1d(num_features)
bn.state_dict()

OrderedDict([('weight', tensor([1., 1., 1., 1.])),
             ('bias', tensor([0., 0., 0., 0.])),
             ('running_mean', tensor([0., 0., 0., 0.])),
             ('running_var', tensor([1., 1., 1., 1.])),
             ('num_batches_tracked', tensor(0))])

In [15]:
bn.train()
torch.manual_seed(0)
for _ in range(100):
    x = torch.randn(batch_size, num_features)
    bn(x)
bn.state_dict()

OrderedDict([('weight', tensor([1., 1., 1., 1.])),
             ('bias', tensor([0., 0., 0., 0.])),
             ('running_mean', tensor([ 0.0527, -0.0417, -0.0876, -0.0537])),
             ('running_var', tensor([0.9879, 0.4089, 1.3890, 0.7146])),
             ('num_batches_tracked', tensor(100))])

In [16]:
bn.eval()
x = torch.randn(1, num_features)
y = bn(x)
y

tensor([[ 0.8423, -1.5081,  1.0178,  2.3462]],
       grad_fn=<NativeBatchNormBackward0>)

同じ値になったね。

`nn.BatchNorm1d`はチャンネル軸が加わった3次元のデータに対しても使用できる。チャンネルごとに正規化を行う。

<br>

ここでのチャンネル軸というのは、サンプルを表した2階以上のテンソルの1番上の軸のこと。CNNで画像を扱うときによく見る。画像以外で使われている場面はあまり見ないが、とりあえず[公式ドキュメント](https://pytorch.org/docs/stable/generated/torch.nn.BatchNorm1d.html)でそう呼ばれていたので倣った。

チャンネル軸が加わった3次元のデータと表現したが、時系列データという解釈もできて、実際に公式ドキュメントでは3つ目の軸を`sequence length`と呼んでいる。

In [17]:
batch_size = 2
c = 3
num_features = 4
x = torch.arange(0, batch_size * c * num_features, dtype=torch.float32)
x = x.reshape(batch_size, c, num_features)
x

tensor([[[ 0.,  1.,  2.,  3.],
         [ 4.,  5.,  6.,  7.],
         [ 8.,  9., 10., 11.]],

        [[12., 13., 14., 15.],
         [16., 17., 18., 19.],
         [20., 21., 22., 23.]]])

In [18]:
bn = nn.BatchNorm1d(c)
y = bn(x)
y

tensor([[[-1.2288, -1.0650, -0.9012, -0.7373],
         [-1.2288, -1.0650, -0.9012, -0.7373],
         [-1.2288, -1.0650, -0.9012, -0.7373]],

        [[ 0.7373,  0.9012,  1.0650,  1.2288],
         [ 0.7373,  0.9012,  1.0650,  1.2288],
         [ 0.7373,  0.9012,  1.0650,  1.2288]]],
       grad_fn=<NativeBatchNormBackward0>)

こういうこと。

In [19]:
mean = x.mean(dim=(0, 2), keepdim=True)
var = x.var(dim=(0, 2), unbiased=False, keepdim=True)
gamma = bn.weight.reshape(1, c, 1)
beta = bn.bias.reshape(1, c, 1)
mean, var

(tensor([[[ 7.5000],
          [11.5000],
          [15.5000]]]),
 tensor([[[37.2500],
          [37.2500],
          [37.2500]]]))

In [20]:
y = (x - mean) / torch.sqrt(var + eps) * gamma + beta
y

tensor([[[-1.2288, -1.0650, -0.9012, -0.7373],
         [-1.2288, -1.0650, -0.9012, -0.7373],
         [-1.2288, -1.0650, -0.9012, -0.7373]],

        [[ 0.7373,  0.9012,  1.0650,  1.2288],
         [ 0.7373,  0.9012,  1.0650,  1.2288],
         [ 0.7373,  0.9012,  1.0650,  1.2288]]], grad_fn=<AddBackward0>)

各チャンネルが2次元の場合は`nn.BatchNorm2d`を使う。また3次元の場合は`nn.BatchNorm3d`を使う。4次元以上は多分ない。
- https://pytorch.org/docs/stable/generated/torch.nn.BatchNorm2d.html
- https://pytorch.org/docs/stable/generated/torch.nn.BatchNorm3d.html

`nn.BatchNorm2d`はCNNでよく使う。先で説明したCNNの場合の動作と同じ動きをする。

In [21]:
batch_size = 32
c, w, h = 3, 224, 224
x = torch.randn(batch_size, c, w, h)
x.shape

torch.Size([32, 3, 224, 224])

In [22]:
bn = nn.BatchNorm2d(c)
bn.state_dict()

OrderedDict([('weight', tensor([1., 1., 1.])),
             ('bias', tensor([0., 0., 0.])),
             ('running_mean', tensor([0., 0., 0.])),
             ('running_var', tensor([1., 1., 1.])),
             ('num_batches_tracked', tensor(0))])

パラメータの数はチャンネルの数と一緒。

In [23]:
bn(x).shape

torch.Size([32, 3, 224, 224])


---

## 層正規化

*Layer Normalization*

Ba, Jimmy Lei, Jamie Ryan Kiros, and Geoffrey E. Hinton. "Layer normalization." arXiv preprint arXiv:1607.06450 (2016).

サンプルごとに正規化し、特徴量ごとにスケーリング・シフトして出力する層。特徴量の数だけパラメータを持つ。また、この層は演算結果がバッチ内の他のデータに依らないため、学習時と推論時で挙動が変わらない（変える必要がない）。

In [24]:
batch_size = 2
d = 5
x = torch.arange(batch_size * d).reshape(batch_size, d).to(torch.float32)
x

tensor([[0., 1., 2., 3., 4.],
        [5., 6., 7., 8., 9.]])

層作成。  
特徴量の形状を与える。その形状と同じ平均と分散がパラメータとなる。

In [25]:
norm = nn.LayerNorm(d)
for params in norm.parameters():
    print(params)

Parameter containing:
tensor([1., 1., 1., 1., 1.], requires_grad=True)
Parameter containing:
tensor([0., 0., 0., 0., 0.], requires_grad=True)


学習データ全体の統計量は持っていない。

In [26]:
norm.state_dict()

OrderedDict([('weight', tensor([1., 1., 1., 1., 1.])),
             ('bias', tensor([0., 0., 0., 0., 0.]))])

正規化。

In [27]:
y = norm(x)
y

tensor([[-1.4142, -0.7071,  0.0000,  0.7071,  1.4142],
        [-1.4142, -0.7071,  0.0000,  0.7071,  1.4142]],
       grad_fn=<NativeLayerNormBackward0>)

以下と同じ。

In [28]:
mean = x.mean(dim=-1, keepdim=True)
var = x.var(dim=-1, keepdim=True, unbiased=False)
((x - mean) / torch.sqrt(var + norm.eps))

tensor([[-1.4142, -0.7071,  0.0000,  0.7071,  1.4142],
        [-1.4142, -0.7071,  0.0000,  0.7071,  1.4142]])

統計量を求める軸を層（特徴量）方向に指定した（`dim=-1`）。

特徴量は何次元でもいい。例えばRNNの場合、以下のようにすれば各バッチ各隠れ状態が正規化される。

In [29]:
batch_size = 2
seq_len = 3
d = 4
x = torch.arange(batch_size * seq_len * d).reshape(batch_size, seq_len, d).to(torch.float32)
x

tensor([[[ 0.,  1.,  2.,  3.],
         [ 4.,  5.,  6.,  7.],
         [ 8.,  9., 10., 11.]],

        [[12., 13., 14., 15.],
         [16., 17., 18., 19.],
         [20., 21., 22., 23.]]])

In [30]:
norm = nn.LayerNorm(d)
for params in norm.parameters():
    print(params)

Parameter containing:
tensor([1., 1., 1., 1.], requires_grad=True)
Parameter containing:
tensor([0., 0., 0., 0.], requires_grad=True)


In [31]:
y = norm(x)
y

tensor([[[-1.3416, -0.4472,  0.4472,  1.3416],
         [-1.3416, -0.4472,  0.4472,  1.3416],
         [-1.3416, -0.4472,  0.4472,  1.3416]],

        [[-1.3416, -0.4472,  0.4472,  1.3416],
         [-1.3416, -0.4472,  0.4472,  1.3416],
         [-1.3416, -0.4472,  0.4472,  1.3416]]],
       grad_fn=<NativeLayerNormBackward0>)

In [32]:
mean = x.mean(dim=-1, keepdim=True)
var = x.var(dim=-1, keepdim=True, unbiased=False)
((x - mean) / torch.sqrt(var + norm.eps))

tensor([[[-1.3416, -0.4472,  0.4472,  1.3416],
         [-1.3416, -0.4472,  0.4472,  1.3416],
         [-1.3416, -0.4472,  0.4472,  1.3416]],

        [[-1.3416, -0.4472,  0.4472,  1.3416],
         [-1.3416, -0.4472,  0.4472,  1.3416],
         [-1.3416, -0.4472,  0.4472,  1.3416]]])